Copies [this workflow](https://github.com/jyu-theartofml/kaggle_quora/blob/master/02_LSTM_2Dense_layers.ipynb)

# Set Up

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.data_utils import get_file

In [ ]:
#create deep copy of data to change
data = deepcopy(data_raw)

In [ ]:
target = data['is_duplicate']

question1 = list(data['question1'])
question2 = list(data['question2'])

print(len(question1))
print(len(question2))
question1[:5]

In [ ]:
#fit tokenizer
tokenizer = Tokenizer(num_words=200000)
tokenizer.fit_on_texts(question1+question2)

In [ ]:
#transform
question1_word_sequences = tokenizer.texts_to_sequences(question1)
question2_word_sequences = tokenizer.texts_to_sequences(question2)
word_index = tokenizer.word_index #unique words in corpus (training and test sets)

print("Words in index: %d" % len(word_index))

In [ ]:
#pad out sentences
q1_data = pad_sequences(question1_word_sequences, maxlen=25)
q2_data = pad_sequences(question2_word_sequences, maxlen=25)

#ensure target is int
labels = np.array(target, dtype=int)
#check shapes
print('Shape of question1 data tensor:', q1_data.shape)
print('Shape of question2 data tensor:', q2_data.shape)
print('Shape of label tensor:', labels.shape)

### Embeddings

[Download glove] 

In [ ]:
embeddings_index = {}
f = open('glove.840B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

### Model

In [ ]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation,GlobalAveragePooling1D,Lambda,Bidirectional
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam, RMSprop
from keras import backend as K

import keras
keras.__version__

import pickle

In [ ]:
from sklearn.cross_validation import train_test_split

X = np.stack((q1_data, q2_data), axis=1)
target = labels

X_train, X_val, y_train, y_val = train_test_split(X, target, test_size=0.25, random_state=126, stratify=target)
Q1_train = X_train[:,0]
Q2_train = X_train[:,1]
Q1_val = X_val[:,0]
Q2_val = X_val[:,1]

In [ ]:
def vec_distance(vects):
    x, y = vects
    return K.sum(K.square(x - y), axis=1, keepdims=True)
#don't use squar root of the sum, it doens't give a good range to feed to the dense layer.

def vec_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)


In [ ]:
from keras.layers.embeddings import Embedding

nb_words=137077+1
max_sentence_len=25
embedding_layer = Embedding(nb_words,300,
        weights=[embedding_matrix],
        input_length=max_sentence_len,trainable=False)
#dont train this layer!

In [ ]:
#build model
lstm_layer =LSTM(128)

sequence_1_input = Input(shape=(max_sentence_len,), dtype='int32')
embedded_sequences_1 = embedding_layer(sequence_1_input)
x1 = lstm_layer(embedded_sequences_1)

sequence_2_input = Input(shape=(max_sentence_len,), dtype='int32')
embedded_sequences_2 = embedding_layer(sequence_2_input)
y1 = lstm_layer(embedded_sequences_2)

distance=Lambda(vec_distance, output_shape=vec_output_shape)([x1, y1])
dense1=Dense(16, activation='sigmoid')(distance)
dense1 = Dropout(0.3)(dense1)

bn2 = BatchNormalization()(dense1)
prediction=Dense(1, activation='sigmoid')(bn2)

model = Model(input=[sequence_1_input, sequence_2_input], output=prediction)

model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',
        optimizer='adam',
        metrics=['acc'])

early_stopping =EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
hist=model.fit([Q1_train, Q2_train], y_train, validation_data=([Q1_val, Q2_val], y_val), verbose=1, 
          nb_epoch=10, batch_size=256, shuffle=True,class_weight=None, callbacks=[early_stopping])
#takes long time to initiate
#using dense() layer and sigmoid activation

### Evaluation

In [ ]:
from keras.models import model_from_json

In [ ]:
#save model and weights

# export model to JSON
model_json = model.to_json()
with open("brnn_model_distance_128_d16_d05.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("brnn_model_distance_128_d16_d05.h5")
print("Saved model to disk")

In [ ]:
# load json and create model
json_file = open('lstm_model_distance_128.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("lstm_model_distance_128.h5")
print("Loaded model from disk")

In [ ]:
pred=model.predict([test1_data, test2_data],verbose=1)